In [5]:
import pandas as pd
import numpy as np
import os
import re

In [11]:
file_path = 'C:/Users/user/Dropbox/korean_war/GIS_control/새마을총람/새마을총람_nm_kor/'

file_ext='xlsx'
save_path=file_path+'python_loop/'

files=os.listdir(file_path)
working_files = [f for f in files if re.search('[\.]'+file_ext+'$', f)]

In [12]:
for f in working_files:
    df= pd.read_excel(file_path+f,sheet_name='data_entry')

    #drop unnecessary rows, columns
    df.rename(columns={"Unnamed: 0":"sido","Unnamed: 1":"sig","Unnamed: 2":"emd"},inplace=True)
    df.drop(df.index[[0,1]],inplace=True)
    df.dropna(how='any',subset=['emd'],inplace=True)
    df.drop(df.columns[[4,5,38]],axis=1,inplace=True)

    #to count the number of villages in each township
    df.insert(4,'num_villages',1) 
    
    #convert strings to numerics
    for k in df.columns:
        if k not in {'sido','sig','emd','code no 1','land_type'}:
            df[k]=pd.to_numeric(df[k],errors='coerce')

    #obtain farmland area for each township
    df.insert(9,'farmland_area',df['farmland_perhousehold']*df['household_total'])

    #to calculate the weighted average of distance by household/population of each villages.
    df['household_sum']=df.groupby(['sig','emd'])['household_total'].transform('sum')
    df['pop_sum']=df.groupby(['sig','emd'])['pop_total'].transform('sum')
    df['weight_by_hh']=df['household_total']/df['household_sum']
    df['weight_by_pp']=df['pop_total']/df['pop_sum']
    df['weighted_distance_hh']=df['distance']*df['weight_by_hh']
    df['weighted_distance_pp']=df['distance']*df['weight_by_pp']

    #generate land type dummies
    df = pd.concat([df, pd.get_dummies(df['land_type'])], axis=1)
    df.drop('land_type',axis=1,inplace=True)

    #replace missing values with zero
    df.replace(np.nan,0,inplace=True)

    #aggregate
    new_dict={k:(np.average if k =='age' else np.sum) for k in df.columns if k not in {'sido','sig','emd','code no 1'}}
    df_agg=df.groupby(['sido','sig','emd'],as_index=False).agg(new_dict)

    #retrieve code no 1
    codeno1=df.groupby(['sido','sig','emd'],as_index=False)['code no 1'].agg(pd.Series.mode)['code no 1']
    df_agg.insert(3,'code no 1',codeno1)

    #dropping farmland_perhousehold, distance, household/pop_sum, weight_by_hh/pop
    k=[10]
    k+=list(range(36,41))
    df_agg.drop(df.columns[k],axis=1,inplace=True) 
    df_agg['age']=df_agg['age'].round(decimals=1) #rounding upto 1st decimal point
    df_agg.insert(10,'farmland_perhousehold',df_agg['farmland_area']/df_agg['household_total'])

    df_agg.to_excel(save_path+f, 
              encoding='utf_8_sig',   
                index=False)   

In [13]:
#append all excel files into one
new_path=file_path+'python_loop/'
all_files=os.listdir(new_path)

append_list=[]
for f in all_files:
    append_list.append(pd.read_excel(new_path+f))
df_app = pd.concat(append_list, ignore_index=True)

df_app.to_excel(new_path+'all_provinces_aggregated.xlsx', 
              encoding='utf_8_sig',   
                index=False)   

In [22]:
#merge with geocontrol data


In [33]:
save_name='merged_emd1970_samaeul.xlsx'
merged_df.to_excel(geo_file_path + save_name,
                   encoding='utf_8_sig',
                   index=False)